# Cleaning the Trip Files

There are a huge amount of trip observations. I make it at about 30,000,000. The guy in the video said a couple million. I can't guarantee my count or his at the moment. The point is that they should perhaps be kept in separate files, but with consistent file and column names. The size of the dataset suggests that `Hadoop` and `Spark` could be helpful, if we decide to keep the entire volume of trip data. Given that there are many `NA`s within some of the files, sampling seems to be a preferable route.

In [2]:
from cs1.trips import *

## Column Names

Since I'm more familiar with `pandas` than any other data manipulation tool, it's the one I'm starting with. My goal is to get all of the data into a `mysql` database and, preferably, to have the table for the trips partitioned by month (and year). The first hurdle to overcome, of course, is the inconsistency in the column names. I'm sure there's some history as to when and why the names were changed, but none of that is really relevant. The main point is just to make sure that they are consistent within the database. The most direct way to do that is to make them consistent in their CSV files.

The `unique-trip_cols` function returns all of the column names found in all of the files.

In [ ]:
columnize(unique_trip_cols())

In [ ]:
consist_cols()

In [ ]:
FILES = list_trip_files()

In [ ]:
len(FILES)

In [ ]:
pp(COLS_LIST)

In [ ]:
df = open_data_frame(FILES[1])

In [ ]:
df

In [ ]:
cd(TEST_DIR)
columnize(list(map(str, unique_values("From Station ID"))))

In [ ]:
unknown_stations = set(unique_values("From Station ID")).union(unique_values("To Station ID")).difference(stations.columns)

In [ ]:
N = len(df)

In [ ]:
len(df['ID'].unique()) == N

In [ ]:
len(df['ID'].dropna()) == N

In [ ]:
df.loc[N - 1, "Bike ID"] - df.loc[0, "Bike ID"] == N

In [ ]:
len(df['Bike ID'].dropna()) == N

In [ ]:
len(df['Bike ID'].unique())

In [ ]:
len(df['Duration'].dropna())

In [ ]:
ERRORS = dict()
ERR_INDEX = list()
for i in COLS_LIST:
    ERRORS[i] = list()

for p in list_trip_files(src=TEST_DIR):
    df = pd.read_csv(str(p))
    ERR_INDEX.append(p.name)
    for c in df.columns:
        ERRORS[c].append(len(df[df[c].isna()]))
    for c in set(COLS_LIST).difference(df.columns):
        ERRORS[c].append(pd.NA)

In [ ]:
errs_df = pd.DataFrame(ERRORS, index=ERR_INDEX)

In [ ]:
errs_df

In [ ]:
errs_df.to_csv(str(BASE_DIR / 'nulls.csv'))

In [ ]:
stations = pd.read_csv(CLEAN_DIR / 'stations.csv')

In [ ]:
help(pd.DataFrame.reindex)

In [ ]:
stations = stations.reindex(stations['ID']).drop(columns=['ID'])

In [ ]:
stations

In [ ]:
stations.columns[0]

In [ ]:
from collections import namedtuple

In [ ]:
def stations_data():
    stations = pd.read_csv(str(CLEAN_DIR / 'stations.csv'))
    stations.index = stations["ID"]
    return stations.drop(columns=["ID"])

In [ ]:
stations = stations_data()

In [ ]:
stations = None

In [ ]:
missing_bikes = 0
START_LAT = "Start Latitude"
START_LNG = "Start Longitude"
FROM_ID = "From Station ID"
FROM_NAME = "From Station Name"

Coordinates = namedtuple('Coordinates', ["Latitude", "Longitude"])

def station_coords(i:int) -> Coordinates:
    global stations
    if stations is None:
        stations = stations_data()
    try:
        return Coordinates(stations["Latitude"][i], stations["Longitude"][i])
    except KeyError:
        print(f"Invalid station ID: {i}")
        return None

NA_START_LATS = df[START_LAT].isna()
NA_FROM_NAMES = df[FROM_NAME].isna()
NA_FROM_IDs   = df[FROM_ID].isna()

for i in df.index:
    if i in NA_START_LATS.index:
        if i in NA_FROM_IDs.index:
            from_station = df[FROM_ID][i]
        elif not i in NA_FROM_NAMES.index:
            from_station = get_station_id(df[FROM_NAME][i])
        else:
            print(f"Bad ride record, missing from station ID and name: {i}")
            break
        coords = station_coords(from_station)
        if not coords:
            print(f"Error getting station coordinates, line {i}.")
            df = df.drop(i)
            break
        df.loc[i, START_LAT] = coords.Latitude
        df.loc[i, START_LNG] = coords.Longitude

In [ ]:
df

In [ ]:
len(df[START_LAT].isna())

In [ ]:
df = df.dropna(subset=['To Station ID'])

In [ ]:
print(f'Number of thefts: {len(df[df["To Station ID"].isna()])}')

In [ ]:
MIN_DATE = datetime(2013, 1, 1, 0, 0, 0)
MAX_DATE = datetime(2022, 1, 31, 0, 0, 0)

In [ ]:
n = 0
for t in df['End Time']:
    d = parse(t)
    if d < MIN_DATE or d > MAX_DATE:
        n += 1

In [ ]:
n

In [ ]:
df["Duration"] = parse()

In [ ]:
df.iloc[2]["Start Time"]

In [ ]:
for i in range(len(df)):
    df["Duration"].iloc[i] = str(parse(df["End Time"].iloc[i]) - parse(df["Start Time"].iloc[i]))

In [ ]:
df

In [ ]:
help(pd.Series.iloc)

In [ ]:
# new_df.to_csv(str(CSV_DIR / trip_files[0].name), header=False, index=False)

In [ ]:
(CSV_DIR / 'header.csv').write_text(','.join(cols_2_keep))

In [ ]:
cd(DATA_DIR)

In [ ]:
for p in [Path(f) for f in glob('*.csv') if f != '']:
    cols_2_keep = COLS_2_KEEP
    cols_2_keep.extend([f'"{s}"' for s in cols_2_keep])
    columnize(cols_2_keep)
    print(f'Processing file: {p.name}')
    df = pd.read_csv(str(p))
    print(f'{df.columns=}')
    cols_2_drop = list(set(df.columns).difference(cols_2_keep))
    print('Columns 2 Drop: ', cols_2_drop)
    df = df.drop(columns=cols_2_drop).reindex(columns=cols_2_keep)
    print('New Columns: ', df.columns)
    df.to_csv(str(CSV_DIR / 'header.csv'), mode='a', header=False, index=False)
    print()

In [ ]:
GIANT_FILE = CSV_DIR / 'header.csv'

In [ ]:
with GIANT_FILE.open() as f:
    print(f.readline())

In [ ]:
cd(CSV_DIR)
CSV_FILES = glob('*.csv')

In [ ]:
CSV_FILES

In [ ]:
OUTPUT_FILE = TEST_DIR / 'trips.csv'
ATTRS_FILE = BASE_DIR / 'header.csv'
OUTPUT_FILE.write_text(ATTRS_FILE.read_text())
print(OUTPUT_FILE.read_text())

In [ ]:
with Path(CSV_FILES[0]).open() as f:
    s = f.readline()
    print(s)

In [ ]:
h = OUTPUT_FILE.read_text()

In [ ]:
h

In [ ]:
OUTPUT_FILE.write_text(h + '\n')

In [ ]:
with OUTPUT_FILE.open() as f:
    s = f.readline().rstrip('\n')
s

In [ ]:
p = Path(CSV_FILES[0])

In [ ]:
p

In [ ]:
with p.open() as f:
    s = f.readline().rstrip('\n')

In [ ]:
s

In [ ]:
s.rstrip('\n')

In [ ]:
len(re.compile(',').findall(s))

In [ ]:
v = s.split(',')

In [ ]:
v

In [ ]:
start, end, origin, dest, user, gender, birth = v

In [ ]:
birth

In [ ]:
start, end, origin, dest, user, gender, birth = s.split(',')
start = str(parse(start))
end = str(parse(end))
origin = int(origin.split('.')[0])
dest = int(dest.split('.')[0])
if birth:
    birth = str(parse(birth).year())


In [ ]:
print(f'{start=}')
print(f'{end=}')
print(f'{origin=}')
print(f'{dest=}')
print(f'{birth=}')

In [ ]:
parse(start)

In [ ]:
str(start)

In [ ]:
cnx, cursor = db_connect()

In [ ]:
HEADER_FILE = BASE_DIR / 'header.csv'
SUBSCRIBER_TYPES = ['Dependent', 'Subscriber', 'member']
CUSTOMER_TYPES = ['Customer', 'casual']
UNIQUE_GENDERS = ['Female', 'Male']
HEADERS = (BASE_DIR / 'header.csv').read_text().rstrip('\n').split(',')

In [ ]:
HEADERS

In [ ]:
p = Path(CSV_FILES[38])
with p.open() as f:
    s = f.readline().rstrip('\n')
    start, end, origin, dest, user, gender, birth = s.split(',')
    start = str(parse(start))
    end = str(parse(end))
    origin = int(origin)
    if user:
        user = 'Subscriber' if user == 'member' else ('Customer' if user == 'casual' else user)
    dest = int(dest)
    if birth:
        birth = str(int(birth.split('.')[0]))
    else:
        birth = 0
    statement = f"""INSERT INTO trips(Start_Time, End_Time, From_Station_ID, To_Station_ID, User_Type, Gender, Birth_Year) VALUES('{start}', '{end}', '{origin}', '{dest}', '{user}', '{gender}', '{birth}');"""
    cursor.execute(statement)
    cnx.commit()

In [ ]:
cnx.commit()

In [ ]:
statement = f"""INSERT INTO Authors (First, Last) VALUES('{names[0]}', '{names[-1]}');"""

In [ ]:
(BASE_DIR / 'header.csv').read_text().rstrip('\n')

In [ ]:
HEADERS = (BASE_DIR / 'header.csv').read_text().rstrip('\n')
HEADERS

In [ ]:
head_frame = pd.read_csv(HEADER_FILE)

In [ ]:
head_frame

In [ ]:
file_frame = pd.read_csv(CSV_FILES[1], header=None)

In [ ]:
file_frame

In [ ]:
file_frame.columns = head_frame.columns
df = pd.concat([head_frame, file_frame])
df

In [ ]:
HEADER_FILE = BASE_DIR / 'header.csv'

def read_trip_csv_frame(f):
    h = pd.read_csv(HEADER_FILE)
    ff = pd.read_csv(f)
    ff.columns = h.columns
    return pd.concat([h, ff])

def unique_values(s):
    v = list()
    for f in CSV_FILES:
        df = read_trip_csv_frame(f)
        for u in df[s].dropna().unique():
            v.append(u)
    v = list(set(v))
    v.sort()
    if '' in v:
        v = v.remove('')
    return v

In [ ]:
df = read_trip_csv_frame(CSV_FILES[4])

In [ ]:
df

In [ ]:
unique_values('User Type')

In [ ]:
unique_values('Gender')

In [ ]:
unique_values('Birth Year')

In [ ]:
STAGED_DIR = BASE_DIR / 'staged'

OUTPUT_FILE = STAGED_DIR / 'trips.csv'

In [ ]:
trip_files

In [ ]:
cd(CSV_DIR)
CSV_FILES = [Path(s) for s in glob('**')]
OUTPUT_FILE.write_text(HEADER_FILE.read_text() + '\n')
n = 0
with OUTPUT_FILE.open(mode='a+') as out:
    for p in CSV_FILES:
        print(p)
        with p.open() as f:
            while True:
                s = f.readline()
                if not s:
                    break
                print(s.rstrip('\n'), file=out)
                n += 1
print(f'{n=}')

In [ ]:
Path(OUTPUT_FILE).read_text()

In [ ]:
CSV_FILES = [Path(s) for s in glob('**')]

In [ ]:
(TEST_DIR / 'trips.csv').write_text(HEADER_FILE.read_text() + '\n' + CSV_FILES[12].read_text())


In [ ]:
def check_trip_files():
    pp(trip_files)

In [ ]:
check_trip_files()

In [ ]:
df = pd.read_csv(CSV_FILES[12])

In [ ]:
df

In [ ]:
CSV_FILES[12].name

In [ ]:
cd(DATA_DIR)

In [ ]:
df = pd.read_csv(CSV_FILES[12].name)

In [30]:
df

,ID,Bike Type,Start Time,End Time,From Station Name,From Station ID,To Station Name,To Station ID,Start Latitude,Start Longitude,End Latitude,End Longitude,User Type,Duration,Distance (ft)
0,46F8167220E4431F,electric_bike,2021-12-07 15:06:07,2021-12-07 15:13:42,Laflin St & Cullerton St,13307,Morgan St & Polk St,TA1307000130,41.854833,-87.663660,41.871969,-87.650965,member,0 days 00:07:35,7773.005521
1,73A77762838B32FD,electric_bike,2021-12-11 03:43:29,2021-12-11 04:10:23,LaSalle Dr & Huron St,KP1705001026,Clarendon Ave & Leland Ave,TA1307000119,41.894405,-87.632331,41.967968,-87.650001,casual,0 days 00:26:54,27575.504022
2,4CF42452054F59C5,electric_bike,2021-12-15 23:10:28,2021-12-15 23:23:14,Halsted St & North Branch St,KA1504000117,Broadway & Barry Ave,13137,41.899357,-87.648522,41.937582,-87.644098,member,0 days 00:12:46,14025.644540
3,3278BA87BF698339,classic_bike,2021-12-26 16:16:10,2021-12-26 16:30:53,Halsted St & North Branch St,KA1504000117,LaSalle Dr & Huron St,KP1705001026,41.899390,-87.648545,41.894877,-87.632326,member,0 days 00:14:43,6136.229807
4,6FF54232576A3B73,electric_bike,2021-12-30 11:31:05,2021-12-30 11:51:21,Leavitt St & Chicago Ave,18058,Clark St & Drummond Pl,TA1307000142,41.895579,-87.682024,41.931248,-87.644336,member,0 days 00:20:16,18913.644997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247535,847431F3D5353AB7,electric_bike,2021-12-12 13:36:55,2021-12-12 13:56:08,Canal St & Madison St,13341,NaN,NaN,41.882289,-87.639752,41.890000,-87.610000,casual,0 days 00:19:13,11202.644790
247536,CF407BBC3B9FAD63,electric_bike,2021-12-06 19:37:50,2021-12-06 19:44:51,Canal St & Madison St,13341,Kingsbury St & Kinzie St,KA1503000043,41.882123,-87.640053,41.889106,-87.638862,member,0 days 00:07:01,2582.040362
247537,60BB69EBF5440E92,electric_bike,2021-12-02 08:57:04,2021-12-02 09:05:21,Canal St & Madison St,13341,Dearborn St & Monroe St,TA1305000006,41.881956,-87.639955,41.880254,-87.629603,member,0 days 00:08:17,3823.945826
247538,C414F654A28635B8,electric_bike,2021-12-13 09:00:26,2021-12-13 09:14:39,Lawndale Ave & 16th St,362.0,NaN,NaN,41.860000,-87.720000,41.850000,-87.710000,member,0 days 00:14:13,5154.651300


In [3]:
L = [p for p in TRIP_FILES if '2021' in p.stem]

In [4]:
df = pd.DataFrame()
for p in L:
    if len(df) == 0:
        df = pd.read_csv(str(p))
    else:
        df = pd.concat([pd.read_csv(str(p))])

In [5]:
df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,99FEC93BA843FB20,electric_bike,2021-06-13 14:31:28,2021-06-13 14:34:11,NaN,NaN,NaN,NaN,41.800000,-87.590000,41.800000,-87.600000,member
1,06048DCFC8520CAF,electric_bike,2021-06-04 11:18:02,2021-06-04 11:24:19,NaN,NaN,NaN,NaN,41.790000,-87.590000,41.800000,-87.600000,member
2,9598066F68045DF2,electric_bike,2021-06-04 09:49:35,2021-06-04 09:55:34,NaN,NaN,NaN,NaN,41.800000,-87.600000,41.790000,-87.590000,member
3,B03C0FE48C412214,electric_bike,2021-06-03 19:56:05,2021-06-03 20:21:55,NaN,NaN,NaN,NaN,41.780000,-87.580000,41.800000,-87.600000,member
4,B9EEA89F8FEE73B7,electric_bike,2021-06-04 14:05:51,2021-06-04 14:09:59,NaN,NaN,NaN,NaN,41.800000,-87.590000,41.790000,-87.590000,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
729590,CB282292CCFCE74F,electric_bike,2021-06-14 00:17:31,2021-06-14 00:56:46,Wells St & Polk St,SL-011,NaN,NaN,41.872412,-87.633715,41.880000,-87.620000,casual
729591,47BD346FAFB9BE6D,classic_bike,2021-06-30 17:35:10,2021-06-30 17:43:20,Clark St & Chicago Ave,13303,Kingsbury St & Kinzie St,KA1503000043,41.896750,-87.630890,41.889177,-87.638506,member
729592,52467C23D17C6AFE,classic_bike,2021-06-13 19:24:30,2021-06-13 19:34:11,Indiana Ave & 26th St,TA1307000005,State St & 19th St,SL-013,41.845687,-87.622481,41.856594,-87.627542,member
729593,7DF6D74420D7D9E6,electric_bike,2021-06-08 15:44:28,2021-06-08 16:15:01,Clark St & Chicago Ave,13303,NaN,NaN,41.896485,-87.630608,41.940000,-87.710000,member


In [43]:
df.to_csv(str(CLEAN_DIR / '2021-cyclistic-rides.csv'))

In [6]:
df["Duration"] = df["End Time"].map(parse) - df["Start Time"].map(parse)

KeyError: 'End Time'

In [ ]:
type(df["Start Latitude"][0])

In [40]:
df["Distance (ft)"] = round(geo_deg_2_feet(sqrt((df["End Latitude"] - df["Start Latitude"])**2 + (df["End Longitude"] - df["Start Longitude"])**2)))

In [2]:
df = pd.read_csv(str(CLEAN_DIR / '2021-cyclistic-rides.csv'))

In [3]:
df.describe()

,Unnamed: 0,Start Latitude,Start Longitude,End Latitude,End Longitude,Distance (ft)
count,247540.000000,247540.000000,247540.000000,247396.000000,247396.000000,247396.000000
mean,123769.500000,41.896822,-87.649813,41.896917,-87.649981,7187.015384
std,71458.787155,0.049759,0.033691,0.049730,0.033747,6546.624380
min,0.000000,41.640000,-87.840000,41.480000,-87.850000,0.000000
25%,61884.750000,41.879434,-87.667178,41.879809,-87.667429,3240.000000
50%,123769.500000,41.895644,-87.644135,41.895748,-87.644140,5238.000000
75%,185654.250000,41.928712,-87.629560,41.928712,-87.629634,9076.000000
max,247539.000000,42.070000,-87.520000,42.070000,-87.520000,77825.000000


In [4]:
df[df["User Type"] == 'member'].drop(columns=["Start Latitude", "Start Longitude", "End Latitude", "End Longitude"]).describe()

,Unnamed: 0,Distance (ft)
count,177802.000000,177781.000000
mean,121341.571028,7039.848915
std,70533.886193,6472.716295
min,0.000000,0.000000
25%,59156.250000,3118.000000
50%,121643.000000,5155.000000
75%,181271.750000,8837.000000
max,247539.000000,69897.000000


In [5]:
df[df["User Type"] == 'casual'].drop(columns=["Start Latitude", "Start Longitude", "End Latitude", "End Longitude"]).describe()

,Unnamed: 0,Distance (ft)
count,69738.000000,69615.000000
mean,129959.677909,7562.845335
std,73402.445970,6717.119711
min,1.000000,0.000000
25%,68458.500000,3638.000000
50%,130339.500000,5760.000000
75%,196713.750000,9716.000000
max,247535.000000,77825.000000


In [53]:
ERRORS = dict()
ERR_INDEX = list()
for i in COLS_LIST:
    ERRORS[i] = list()
ERRORS["Distance (ft)"] = list()
ERRORS["Duration"] = list()

# df = pd.read_csv(str(p))
# ERR_INDEX.append(p.name)
for c in df.columns:
    ERRORS[c].append(len(df[df[c].isna()]))
for c in set(COLS_LIST).difference(df.columns):
    ERRORS[c].append(pd.NA)

In [54]:
nulls = pd.DataFrame(ERRORS)

In [55]:
nulls

,ID,Start Time,End Time,Bike ID,Duration,From Station ID,To Station ID,User Type,Gender,Birth Year,To Station Name,From Station Name,End Latitude,End Longitude,Start Latitude,Start Longitude,Bike Type,Distance (ft)
0,0,0,0,<NA>,0,51063,53354,0,<NA>,<NA>,53354,51063,0,0,0,0,0,0


In [52]:
df = df.drop(df[df["End Latitude"].isna()].index)

In [7]:
consist_cols()